# Decision Trees

Like SVMs, decision trees are versatile machine learning algorithms that can perform classification & regression tasks, & even multioutput tasks. They are powerful algorithms, capable of fitting complex datasets.

Decision trees are also the fundamental components of random forests, which are among the most powerful machine learning algorithm available today.

In this lesson, we will discuss how to train, visualise, & make predictions with decision trees. Then we will go through the CART training algorithm used by scikit-learn & discuss how we regularise trees & use them for regression tasks. Finally, we will discuss some of the limitations of decision trees.

---

# Training & Visualising a Decision Tree

To understand decision trees, let's build one & take a look at how it makes predictions. The following code trains a `DecisionTreeClassifier()` on the iris dataset.

In [2]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X = iris.data[:, 2:] # petal length & width
y = iris.target

tree_classifier = DecisionTreeClassifier(max_depth = 2)
tree_classifier.fit(X, y)

DecisionTreeClassifier(max_depth=2)

You can visualise the train decision tree by first using the `export_graphviz()` method to output a graph definition file called `iris_tree.dot`:

In [3]:
from graphviz import Source
from sklearn.tree import export_graphviz

export_graphviz(tree_classifier, out_file = "Images/iris_tree.dot",
                feature_names = iris.feature_names[2:], 
                class_names = iris.target_names, rounded = True, filled = True)

Then you can use the dot command-line tool from the graphviz package to convert this *.dot* file to a variety of formats, such as PDF or PNG. This command line converts the *.dot* file to a *.png* image file:

**dot -Tpng iris_tree.dot -o iris_tree.png** 

Your first decision tree looks like this.

<img src = "Images/iris_tree.png" width = "350" style = "margin:auto"/>

---

# Making Predictions

Let's see how the tree makes predictions. Suppose you find an iris flower & you want to classify it. You start at the *root node* (depth 0, at the top): this node asks whether the flower's petal length is smaller than 2.45cm. If it is, then you move down to the root's left child node (depth 1, left). In this case, it is a *leaf node* (i.e., it does not have any child nodes), so it does not ask any questions: simply look at the predicted class for that node, & the decision tree predicts that your flower is an *Iris setosa* (`class = setosa`).

Now suppose you find another flower, & this time, the petal length is greater than 2.45cm. You must move down to the root's right child node (depth 1, right), which is not a leaf node, so the node asks another question: is the petal width smaller than 1.75cm? If it is, then your flower is most likely an *Iris versicolor* (depth 2, left). If not, it is likely an *Iris virginica* (depth 2, right). It's really that simple.

A node's `samples` attributes counts how many training instances it applies to. For example, 100 training instances have a petal length greater than 2.45cm (depth 1, right), & of those 100, 54 have a petal width smaller than 1.75cm (depth 2, left). A node's `value` attribute tells you how many training instances of each class this node applies to: for example, the bottom-right node applies to 0 *Iris setosa*, 1 *Iris versicolour*, 45 *Iris virginica*. Finally, a node's `gini` attribute measures its *impurity*: a node is "pure" (`gini = 0`) if all training instances it applies to belong to the same class. For example, since the depth 1 left node applies only to *Iris setosa* training instance, it is pure & its `gini` score is 0. The below equation shows how the training algorithm computes the `gini` score $G_i$ of the $i^{th}$ node. The depth 2 left node has a `gini` score equal to $1 - (0/54)^2 -  (49/54)^2 - (5/54)^2 \approx 0.168$.

$$G_i = 1 - \sum^{n}_{k = 1} {p_{i, k}}^2$$

In this equation:

* $p_{i, k}$ is the ratio of class *k* instances among the training instances in the $i^{th}$ node.

<img src = "Images/Decision Tree Decision Boundary.png" width = "600" style = "margin:auto"/>

The above figure shows this decision tree's decision boundaries. The thick vertical line represents the decision boundary of the root node (depth 0): petal length = 2.45cm. Since the left hand area is pure (only *Iris setosa*), it cannot be split any further. However, the right hand area is impure, so the depth 1 right node splits it at petal width = 1.75cm (represented by the dashed line). Since the `max_depth` was set to 2, the decision tree stops right there. If you set `max_depth` to 3, then the two depth 2 nodes would each add another decision boundary (represented by the dotted lines).

---

# Estimating Class Probabilities

A decision tree can also estimate the probability that an instance belongs to a particular class *k*. First, it traverses the tree to find the leaf node for this instance, & then it returns the ratio of training instances of class *k* in this node. For example, suppose you have found a flower whose petals are 5cm long & 1.5cm wide. The corresponding leaf node is the depth 2 left node, so the decision tree should output the following probabilities: 0% for *Iris setosa* (0/54), 90.7% for *Iris versicolor* (49/54), & 9.3% for *Iris virginica* (5/54). & if you ask it to predict the class, it should output *Iris versicolour* (class 1) because it has the highest probability.

In [4]:
tree_classifier.predict_proba([[5, 1.5]])

array([[0.        , 0.90740741, 0.09259259]])

In [5]:
tree_classifier.predict([[5, 1.5]])

array([1])

Notice that the estimated probabilities would be identical anywhere else in the bottom-right rectangle -- for example, if petals were 6cm long & 1.5cm wide (even though it seems obvious that it would most likely be an *Iris virginica* in this case).

---

# The CART Training Algorithm

Scikit-learn uses the *Classification & Regression Tree* (CART) algorithm to train decision trees (also called "growing" trees). The algorithm works by first splitting the training set into two subsets using a single feature *k* & a threshold $t_k$ (e.g., "petal length $\leq$ 2.45 cm"). How does it choose *k* & $t_k$? It searchs for the pair (*k*, $t_k$) that produces the purest subsets (weighted by their size). The below function gives the cost function that the CART algorithm tries to minimise.

$$\begin{split}
J(k, t_k) = \frac{m_{left}}{m}G_{left} + \frac{m_{right}}{m}G_{right} \\
where \Biggl\{ \begin{split}
G_{left/right} \quad measures\ the\ impurity\ of\ the\ left/right\ subset, \\
m_{left/right} \quad is\ the\ number\ of\ instances\ in\ the\ left/right\ subset.
\end{split}
\end{split}$$

Once the CART algorithm has successfully split the training set in two, it splits the subsets using the same logic, then the sub-subsets, & so on, recursively. It stops the recursing once it reaches the maximum depth (defined the by `max_depth` hyperparameter), or if it cannot find a split that will reduce impurity. A few other hyperparameters control additional stopping conditions (`min_samples_split`, `min_samples_leaf`, `min_weight_fraction_leaf`, & `max_leaf_nodes`).

---

# Computational Complexity

Making predictions requires traversing the decision tree from the root to a leaf. Decision trees generally are approximately balanced, so traversing the decision tree requires going through rougly $O(log_2(m))$ nodes. Since each node only requires checking the value of one feature, the overall prediction complexity is $O(log_2(m))$, independent of the number of features. So predictions are very fast, even when dealing with large training sets.

The training algorithm compares all features (or less if `max_features` is set) on all samples at each node. Comparing all features on all samples at each node results in a training complexity of $O(n * m log_2(m))$. For small training sets (less than a few thousand instances), scikit-learn can speed up training by presorting the data (set `presort = True`), but doing that slows down training considerably for larger training sets.

---

# Gini Impurity or Entropy

By default, the gini impurity measure is used, by you can select the *entropy* measure instead by setting the *criterion* hyperparameter to `"entropy"`. The concept of entropy originated in thermodynamics as a measure of molecular disorder: entropy approaches zero when molecules are still & well ordered. Entropy later spread to a wide variety of domains, including Shannon's *information theory*, where it measures the average information content of a message: entropy is zero when all messages are identical. In machine learning, entropy is frequently used as an impurity measure: a set's entropy is zero when it contains instances of only one class. The below equation shows the definition of the entropy of the $i^{th}$ node. For example, the depth 2 left node has an entropy equal to $-(49/54)log_2(49/54) - (5/54)log_2(5/54) \approx 0.445$.

$$H_i = \underset{p_{i,k \neq 0}}{-\sum^{n}_{k = 1}} p_{i, k}log_2(p_{i, k})$$

So, should you use gini impurity or entropy? The truth is, most of the time, it does not make a big difference: they lead to similar trees. Gini impurity is slightly faster to compute, so it is a good default. However, when they differ, gini impurity tends to isolate the most frequent class in its own branch of the tree, while entropy tends to produce slightly more balanced trees.

---

# Regularisation Hyperparameters

 Decision trees make very few assumptions about the training data (as opposed to linear models, which assume that the data is linear, for example). If left unconstrained, the tree structure will adapt itself to the training data, fitting it very closesly -- indeed, most likely overfitting it. Such a model is often called a *nonparametric model* not because it does not have any parameters (it often has a lot), but because the number of parameters is not determined prior to  training, so the model is free to stick closely to the data. In contrast, a *parametric model*, such as a linear model, has predetermined number of parameters, so its degree of freedom is limited, reducing the risk of overfitting (but increasing the risk of underfitting).
 
To avoid overfitting the training data, you need to restrict the decision tree's freedom during training. As you know by now, this is called regularisation. The regularisation hyperparameters depend on the algorithm used, but generally you can at least restrict the maximum depth of the decision tree. In Scikit-learn, this is controlled by the `max_depth` hyperparameter (the default value is `None`, which means unlimited). Reducing `max_depth` will regularise the model & thus reduce the risk of overfitting.

The `DecisionTreeClassifier()` class has a few other parameters that similarly restrict the shape of the decision tree: `min_samples_split` (the minimum number of samples a node must have before it can be split), `min_samples_leaf` (the minimum number of samples a leaf node must have), `min_weight_fraction_leaf` (same as `min_samples_leaf` but expressed as a fraction of the total number of weighted instances), `max_leaf_nodes` (the maximum number of leaf nodes), & `max_features` (the maximum number of features that are evaluated for splitting at each node). Increasing `min_*` hyperparameters or reducing `max_*` hyperparameters will regularise the model.

The below figure shows two decision trees trained on the moons dataset. On the left, the decision tree is trained with the default hyperparameters (i.e., no restrictions), & on the right, it's trained with `min_samples_leaf = 4`. It is quite obvious that the model on the left is overfitting, & the model on the right will probably generalise better.

<img src = "Images/Decision Tree Regularisation.png" width = "600" style = "margin:auto"/>

---

# Regression

Decision trees are also capable of performing regression tasks. Let's build a regression tree using scikit-learn's `DecisionTreeRegressor()` class, training it on a noisy quadratic dataset with `max_depth = 2`.

In [6]:
from sklearn.tree import DecisionTreeRegressor
import numpy as np

np.random.seed(42)
m = 200
X = np.random.rand(m, 1)
y = 4 * (X - 0.5) ** 2
y = y + np.random.randn(m, 1) / 10

tree_reg = DecisionTreeRegressor(max_depth = 2)
tree_reg.fit(X, y)

export_graphviz(tree_reg, out_file = "Images/regression_tree.dot",
                feature_names = ["x1"], rounded = True, filled = True)

The resulting tree is represented here:

<img src = "Images/regression_tree.png" width = "500" style = "margin:auto"/>

This tree looks very similar to the classification tree we build earlier. The main difference is that instead of predicting a class in each node, it predicts a value. For example, suppose you want to make a prediction for a new instance with $x_1 = 0.6$. You traverse the tree starting at the root, & you eventually reach the leaf node that predicts `value = 0.111`. This prediction is the average target value of 110 training instances associated with this leaf node, & it results in a mean squared error equal to 0.015 over these 110 instances. 

This model's predictions are represented in the figure below.

<img src = "Images/Two Decision Tree Regressors.png" width = "600" style = "margin:auto"/>

Notice how the predicted for each region is always the average target value of the instances in that region. The algorithm splits each region in a way that makes most training instances as close as possible to that predicted value.

The CART algorithm works mostly the same way as earlier, except that instead of trying to split the training set in a way that minimises impurity, it now tries to split the training set in a way that minimises MSE. The below equation shows the cost function that the CART algorithm tries to minimise.

$$J(k, t_k) = \frac{m_{left}}{m} MSE_{left} + \frac{m_{right}}{m} MSE_{right} \quad where \Biggl\{\begin{split}
MSE_{node} = \sum_{i\ \in\ node} (\hat{y}_{node} - y^{(i)})^2 \\
\hat{y}_{node} = \frac{1}{m_{node}} \sum_{i\ \in\ node} y^{(i)}
\end{split}$$

Just like for classification tasks, decision trees are prone to overfitting when dealing with regression tasks. Without any regularisation (i.e., using the default hyperparameters), you get the predictions on the left in the below figure. These predictions are obviously overfitting the training set very badly, Just setting `min_samples_leaf = 10` results in a much more reasonable model, represented on the right.

<img src = "Images/Decision Tree Regression Regularisation.png" width = "600" style = "margin:auto"/>

---

# Instability

Hopefully by now, you are convinced that decision trees have a lot going for them: they are simple to understand & interpret, easy to use, versatile, & powerful. However, they do have a few limitations. First, as you may have noticed, decision trees love orthogonal decision boundaries (all splits are perpendicular to an axis), which makes them sensitive to training set rotation. The below figure shows a simple linearly separable dataset. On the left, a decision tree can split it easily, while on the right, after the dataset is rotated by $45^{\circ}$, the decision boundary looks unnecessarily convoluted. Although both decision trees fit the training set perfectly, it is very likely that the model on the right will not generalise well. One way to limit this problem is by using principal component analysis, which often results in a better orientation of the training data.

<img src = "Images/Training Set Rotation.png" width = "600" style = "margin:auto"/>

More generally, the main issue with decision trees is that they are very sensitive to small variations in the training data. For example, if you just remove the widest *Iris versicolor* from the training set (the one with petals 4.8cm long & 1.8cm wide) & train a new decision tree, you may get the model represented below (bottom). As you can see, it looks very different from the previous decision tree (top). Actually, since the training algorithm used by scikit-learn is stochastic, you may get very different models even on the same training data (unless you set the `random_state` hyperparameter).

<img src = "Images/Decision Tree Decision Boundary.png" width = "500" style = "margin:auto"/>

<img src = "Images/Training Set Sensitivity.png" width = "500" style = "margin:auto"/>

Random forests can limit this instability by averaging predictions over many trees, as we will see in the next lesson.